In [15]:
import numpy as np
import pandas as pd
from scipy.stats import expon
from scipy import stats
from itertools import combinations as combowombo
import matplotlib.pyplot as plt

In [2]:
with open('Data/d_quite_big.in') as file:
    data = file.read().split()

data = [int(_) for _ in data]
MAXSLICES = data[0]
TYPES = data[1]
PIZZAS = data[2:]
pizza_indices = {pizza: index for index, pizza in enumerate(PIZZAS)}

In [3]:
def create_sorted_prob_list(size = 1, dist = np.random.uniform, **kwargs):
    prob_list = sorted(dist(size = size, **kwargs))
    total = sum(prob_list)
    prob_list = prob_list/total
    return prob_list

In [4]:
def expon_list(size = 1, scale = 1):
    prob_list = sorted(np.array([(1/scale)*expon.pdf(x/scale) for x in range(size)]))
    normed_list = prob_list/sum(prob_list)
    return normed_list

In [5]:
def all_slices(pizza_list):
    possible_slices = sorted([sum(comb) for k in range(1,len(pizza_list)+1) for comb in combowombo(pizza_list,k)])
    return possible_slices

In [6]:
def compute_slices(max = MAXSLICES, pizza_list = PIZZAS):
    final_list = []
    while (max >= 0):
      admissible_pizzas = [pizza for pizza in pizza_list if pizza <= max]
      if admissible_pizzas:
        num_pizzas = len(admissible_pizzas)
        prob_list = create_sorted_prob_list(size=num_pizzas, dist=np.random.exponential, scale = 0.0001)
        index = np.random.choice(num_pizzas,p=prob_list)
        rand_pizza = admissible_pizzas.pop(index)
        pizza_list = admissible_pizzas
        max = max - rand_pizza
        final_list.append(rand_pizza)
      else:
        break
    return final_list

In [16]:
import time
start = time.time()

MAX = 385749466
best_output = []
best_total = 0

if MAX < 5E6:
    reps = 100000
elif MAX < 50E6:
    reps = 10000
elif MAX < 500E6:
    reps = 1000
else:
    reps = 100

#I collected some data on how long a particular best solution survives for and modeled it with a Weibull distribution.
#I computed the relative liklihood that the best solution would survive 42 more rounds given it survived until time T.
#If a solution survives 420 rounds, there is an 78% chance that it will survive 42 more rounds. This was good enough for me to call it
#a winner early. This avoids situations where it is actually impossible to make the MAXSLICES. Yes, I did all this for a joke, but also,
#It's efficient! Ish.

#record_prob = []
counter = 0

for i in range(reps):
    if best_total == MAX:
        break
    if counter >= 420:
        break
    output = compute_slices(max = MAX)
    total = sum(output)
    if total > best_total:
        best_total = total
        best_output = output
        #if counter:
        #    record_prob.append(counter)
        counter = 0
    else:
        counter += 1

print(f'Total = {best_total} \ndiff = {MAX - best_total}')
print(f'It took {time.time()-start} seconds!')

Total = 385749466 
diff = 0
It took 15.085638284683228 seconds!


In [ ]:
Mama_Mia_Its_A_Pizza = sorted([pizza_indices[p] for p in best_output])

In [ ]:
collected_prob = []

In [ ]:
collected_prob.extend(record_prob)
td = pd.DataFrame(collected_prob)
td.plot.hist(bins=30)

In [ ]:
len(collected_prob)

In [ ]:
c = stats.weibull_min.fit(collected_prob)
c

In [ ]:
x = np.linspace(0,1000, 100)
W = stats.weibull_min(*c)
plt.plot(x,W.pdf(x))
